# McStas Absorption logger components
We have now seen how the McStas Union components provide a powerful engine for simulation of complex physics and geometry. 

| Component class         | Links to                       | Purpose                                    |
| :---                    |    :----                       |    :----                                   |
| Process components      |                                | Describe scattering process                |
| Union_make_material     | List of process components     | Collect process and absorption to material |
| Geometry components     | Material definition            | Place a geometry with selected material    |
| Union_master            | Geometries (automatic linking) | Simulation engine                          |
| Union_init / Union_stop |                                | Needed in McStas 3.X at start / end        |

The complexity of the results can however make it difficult to understand the results. This presentation introduces the concept of a absorption logger component which will help us understand!

### Why we can't use regular monitors
We can't use regular monitors as the entire simulation happens in a single component, the master.

![alt](figures/power_point_figures/Slide04.png)

## Absorption loggers
Absorption loggers are a class of Union components, meaning they all behave in a similar way. Whenever a ray is propagated within a volume with a absorption cross section, the Union master reduces the ray weight accordingly. The absorption logger component get the state of the ray at randomly sampled absorption points on the rays path.

Absorption loggers record that information in some way, and differ in what kind of information is stored. It is fairly easy to contribute such a component if what you would like to record is not yet present.

| Abs_logger component                    | Dimensions                                |
| :---                                    |    :----                                  |
| Union_abs_logger_1D_space               | 1D space                                  |
| Union_abs_logger_1D_space_event         | 2D space in event format                  |
| Union_abs_logger_1D_space_tof           | 1D space and time                         |
| Union_abs_logger_1D_space_tof_to_lambda | 1D space and time converted to wavelength |
| Union_abs_logger_2D_space               | 2D space                                  |
| Union_abs_logger_event                  | Events  (position, velocity, weight)      |

## Our example with a diamond sphere in a aluminium shell

In [1]:
import mcstasscript as ms
instrument = ms.McStas_instr("Union_demo", input_path="run_folder")
instrument.settings(output_path="data_folder/union_demo")

instrument.add_component("init", "Union_init")

Al_incoherent = instrument.add_component("Al_incoherent", "Incoherent_process")
Al_incoherent.set_parameters(sigma=4*0.0082, unit_cell_volume=66.4)

Al_powder = instrument.add_component("Al_powder", "Powder_process")
Al_powder.reflections = '"Al.laz"'

Al = instrument.add_component("Al", "Union_make_material")
Al.process_string = '"Al_incoherent,Al_powder"'
Al.my_absorption = 100*4*0.231/66.4

process = instrument.add_component("diamond_incoherent", "Incoherent_process")
process.set_parameters(sigma=8*0.001, unit_cell_volume=45.39)

instrument.add_parameter("diamond_phi", value=0, comment="Rotation of diamond lattice around z [deg]")
crystal_process = instrument.add_component("diamond_crystal", "Single_crystal_process")
crystal_process.set_parameters(mosaic=20, reflections='"C_diamond.lau"',
                               ax=3.567, by=3.567, cz=3.567,)
crystal_process.set_ROTATED([0, 0, "diamond_phi"])


diamond = instrument.add_component("diamond", "Union_make_material")
diamond.process_string = '"diamond_incoherent,diamond_crystal"'
diamond.my_absorption = 100*8*0.0035/45.39


sample_position = instrument.add_component("sample_position", "Arm")
sample_position.set_AT([0,0,1])

instrument.add_parameter("A3", value=0, comment="Rotation of sphere around y [deg]")
sphere = instrument.add_component("sphere", "Union_sphere")
sphere.radius = 0.01
sphere.material_string = '"diamond"'
sphere.priority = 10
sphere.set_AT(0, RELATIVE=sample_position)
sphere.set_ROTATED([0, "A3", 0], RELATIVE=sample_position)

shell = instrument.add_component("shell", "Union_cylinder")
shell.set_parameters(radius=0.05, yheight=0.12,
                     material_string = '"Al"', priority=3,
                     p_interact=0.2)
shell.set_AT(0, RELATIVE=sample_position)

shell_vacuum = instrument.add_component("shell_vacuum", "Union_cylinder")
shell_vacuum.set_parameters(radius=shell.radius - 0.01, yheight=shell.yheight - 0.01,
                            material_string = '"Vacuum"', priority=5)
shell_vacuum.set_AT([0, 0, 0], RELATIVE=sample_position)

source = instrument.add_component("source", "Source_div")
source.set_parameters(focus_aw=0.3, focus_ah=0.3,
                      xwidth=0.005, yheight=0.005,
                      flux=1E12)
source.set_AT(-1, RELATIVE=sample_position)

source.lambda0 = instrument.add_parameter("wavelength", value=5.0,
                                          comment="Wavelength in [Ang]")
source.dlambda = instrument.add_parameter("wavelength_half_width", value=4.5,
                                          comment="Wavelength half width in [Ang]")

master = instrument.add_component("master", "Union_master", RELATIVE=sample_position)
stop = instrument.add_component("stop", "Union_stop")

The following components are found in the work_directory / input_path:
     Union_sphere.comp
     Texture_process.comp
     Union_cone.comp
     Union_stop.comp
     Union_box.comp
     Single_crystal_process.comp
     Union_abs_logger_2D_space.comp
     Union_logger_2D_kf.comp
     Union_init.comp
     Template_process.comp
     PhononSimple_process.comp
     Union_conditional_standard.comp
     Union_abs_logger_1D_space.comp
     Union_abs_logger_event.comp
     Union_abs_logger_1D_space_event.comp
     Non_process.comp
     Union_abs_logger_1D_space_tof.comp
     Union_logger_2D_space.comp
     Union_conditional_PSD.comp
     Union_master.comp
     AF_HB_1D_process.comp
     Union_logger_2D_kf_time.comp
     Union_master_GPU.comp
     Union_cylinder.comp
     Union_abs_logger_1D_space_tof_to_lambda.comp
     Powder_process.comp
     Union_make_material.comp
     Incoherent_process.comp
     Union_logger_1D.comp
     Union_logger_3D_space.comp
     IncoherentPhonon_process.comp
    

## Add an absorption logger
Abs loggers need to be placed in space as that will be their coordinate system for observing absorption. 

In [2]:
abs_logger_zx = instrument.add_component("abs_logger_space_zx", "Union_abs_logger_2D_space", before=master)
abs_logger_zx.set_AT(0, RELATIVE=sample_position)
abs_logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.06, D1_max=0.06, n1=300,
                             D_direction_2='"x"', D2_min=-0.06, D2_max=0.06, n2=300,
                             filename='"abs_logger_zx.dat"')

In [3]:
print(abs_logger_zx)

COMPONENT abs_logger_space_zx = Union_abs_logger_2D_space(
  D_direction_1 = "z", // [string]
  D1_min = -0.06, // [m]
  D1_max = 0.06, // [m]
  n1 = 300, // [1]
  D_direction_2 = "x", // [string]
  D2_min = -0.06, // [m]
  D2_max = 0.06, // [m]
  n2 = 300, // [1]
  filename = "abs_logger_zx.dat" // [string]
)
AT (0, 0, 0) RELATIVE sample_position


### Add absorption loggers for zy and xy plane as well

In [4]:
abs_logger_zy = instrument.add_component("abs_logger_space_zy", "Union_abs_logger_2D_space", before=master)
abs_logger_zy.set_AT(0, RELATIVE=sample_position)
abs_logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.06, D1_max=0.06, n1=300,
                             D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                             filename='"abs_logger_zy.dat"')

abs_logger_xy = instrument.add_component("abs_logger_space_xy", "Union_abs_logger_2D_space", before=master)
abs_logger_xy.set_AT(0, RELATIVE=sample_position)
abs_logger_xy.set_parameters(D_direction_1='"x"', D1_min=-0.06, D1_max=0.06, n1=300,
                             D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                             filename='"abs_logger_xy.dat"')

## Add some normal monitors

In [5]:
instrument.add_declare_var("double", "t_min")
instrument.add_declare_var("double", "t_max")

instrument.append_initialize("t_min = (wavelength - wavelength_half_width)*(1.0 - 0.18 + 0.5)/(K2V*2*PI);")
instrument.append_initialize("t_max = (wavelength + wavelength_half_width)*(1.0 + 0.28 + 0.5)/(K2V*2*PI);")

instrument.add_declare_var("char", "options", array=256)
instrument.append_initialize('sprintf(options,"banana, theta limits=[-180,180] bins=180, t limits=[%g %g] bins=300",t_min,t_max);')

banana_detector = instrument.add_component("banana_tof_detector", "Monitor_nD")
banana_detector.set_RELATIVE(sample_position)
banana_detector.xwidth = 1
banana_detector.yheight = 0.2
banana_detector.restore_neutron = 1
options = "options"
banana_detector.options = options
banana_detector.filename = '"tof_b.dat"'

banana_detector = instrument.add_component("banana_detector", "Monitor_nD")
banana_detector.set_RELATIVE(sample_position)
banana_detector.set_parameters(xwidth = 1.0, yheight = 0.2, filename='"banana.dat"', restore_neutron=1,
                               options='"banana, theta limits=[-180,180] bins=180"')

L_monitor = instrument.add_component("L_monitor", "L_monitor")
L_monitor.set_AT(0.3, RELATIVE=sample_position)
L_monitor.set_parameters(xwidth=0.3, yheight=0.3,
                         Lmin="wavelength - wavelength_half_width",
                         Lmax="wavelength + wavelength_half_width",
                         nL=300, filename='"l_mon.dat"', restore_neutron=1)

## Run a simulation with our absorption loggers

In [6]:
import mcstasscript.jb_interface as ms_widget
%matplotlib widget
ms_widget.show(instrument)

# Lessons learnt from simulation
- Absorption loggers can provide insight into what happens in a Union simulation
- Multiple scattering very wavelength dependent
- There is still much to understand

## Parameters in a absorption logger
Lets look at the parameters available in a absorption logger

- order_total: Show only rays that have scattered this number of times
- order_volume: Show only rays that have scattered this number of times in certain geometry
- target_geometry: Only show absorption in this (these) geometries

In [7]:
abs_logger_zx.show_parameters()

 ___ Help Union_abs_logger_2D_space _________________________________________________
|optional parameter|required parameter|default value|user specified value|
target_geometry = "NULL" [string] // Comma separated list of geometry names 
                                     that will be logged, leave empty for all 
                                     volumes (even not defined yet) 
D_direction_1 = "z" [string] // Direction for first axis ("x", "y" or "z")
D1_min = -0.06 [m] // Histogram boundary, min position value for first axis
D1_max = 0.06 [m] // Histogram boundary, max position value for first axis
n1 = 300 [1] // Number of bins for first axis
D_direction_2 = "x" [string] // Direction for second axis ("x", "y" or "z")
D2_min = -0.06 [m] // Histogram boundary, min position value for second axis
D2_max = 0.06 [m] // Histogram boundary, max position value for second axis
n2 = 300 [1] // Number of bins for second axis
filename = "abs_logger_zx.dat" [string] // Filename of produced da

## Add more abs loggers with these features
Could be a good place to use copy component.

In [8]:
abs_logger_zy = instrument.add_component("abs_logger_space_zy_order_0", "Union_abs_logger_2D_space", before=master)
abs_logger_zy.set_AT(0, RELATIVE=sample_position)
abs_logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.06, D1_max=0.06, n1=300,
                             D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                             filename='"abs_logger_zy_order_0.dat"', order_total=0)

abs_logger_zy = instrument.add_component("abs_logger_space_zy_order_1", "Union_abs_logger_2D_space", before=master)
abs_logger_zy.set_AT(0, RELATIVE=sample_position)
abs_logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.06, D1_max=0.06, n1=300,
                             D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                             filename='"abs_logger_zy_order_1.dat"', order_total=1)

abs_logger_zy = instrument.add_component("abs_logger_space_zy_order_2", "Union_abs_logger_2D_space", before=master)
abs_logger_zy.set_AT(0, RELATIVE=sample_position)
abs_logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.06, D1_max=0.06, n1=300,
                             D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                             filename='"abs_logger_zy_order_2.dat"', order_total=2)

abs_logger_zy = instrument.add_component("abs_logger_space_zy_shell", "Union_abs_logger_2D_space", before=master)
abs_logger_zy.set_AT(0, RELATIVE=sample_position)
abs_logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.06, D1_max=0.06, n1=300,
                             D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                             filename='"abs_logger_zy_shell.dat"',
                             target_geometry='"shell"')

abs_logger_zy = instrument.add_component("abs_logger_space_zy_sample", "Union_abs_logger_2D_space", before=master)
abs_logger_zy.set_AT(0, RELATIVE=sample_position)
abs_logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.06, D1_max=0.06, n1=300,
                             D_direction_2='"y"', D2_min=-0.07, D2_max=0.07, n2=300,
                             filename='"abs_logger_zy_sample.dat"',
                             target_geometry='"sphere"')

### Run simulation again

In [9]:
ms_widget.show(instrument)

# Using absorption loggers to simulate detectors
One of the main purposes of absorption loggers is to simulate detectors!

- Aluminium shell
- He3 internal gas
- Absorption logger on He3 gas
- Maybe wire and other details!

| Abs_logger component                    | Dimensions                                |
| :---                                    |    :----                                  |
| Union_abs_logger_1D_space               | 1D space                                  |
| Union_abs_logger_1D_space_event         | 2D space in event format                  |
| Union_abs_logger_1D_space_tof           | 1D space and time                         |
| Union_abs_logger_1D_space_tof_to_lambda | 1D space and time converted to wavelength |
| Union_abs_logger_2D_space               | 2D space                                  |
| Union_abs_logger_event                  | Events  (position, velocity, weight)      |

![alt](figures/power_point_figures/Slide03.png)

### Simple model of He3 detector tube
<table><tr>
<td> <img src="figures/power_point_figures/Slide3.png" alt="Detector sketch" style="width: 500px;"/> </td>
<td> <img src="figures/output_absorption_point.gif" alt="Animation" style="width: 900px;"/> </td>
</tr></table>

# Union overview so far
We now have 6 different classes of Union components!

| Component class         | Links to                       | Purpose                                    |
| :---                    |    :----                       |    :----                                   |
| Process components      |                                | Describe scattering process                |
| Union_make_material     | List of process components     | Collect process and absorption to material |
| Geometry components     | Material definition            | Place a geometry with selected material    |
| Abs_logger components   | Optionally geometry / process  | Record absorption in Union system          |
| Union_master            | Geometries (automatic linking) | Simulation engine                          |
| Union_init / Union_stop |                                | Needed in McStas 3.X at start / end        |



## Quiz and exercise
The next quiz is exercise 4 which concerns a simple SANS instrument with a He3 detector built with Union components.